# Fitting a Morse Diatomic Absorption spectrum with a non-Condon Moment

In these spectroscopy calculations, we are given $\omega_e$, $\chi_e \omega_e$, the reduced mass $\mu$ and the equilibrium position $r_e$.  For each atom, we want to create a system of units out of these.

\begin{align}
    h &= A \cdot e_u\cdot T_u = A \cdot m_u \frac{l_u^2}{T_u}
\end{align}
lower case means we are setting it still, capital letters mean they are determined.  If we assume right now we want to set $E_u$ to be some spectoscopic value in wavenumbers and set $\hbar$ then we know we have to let time float, which is fine since this code is not a time-dependent one.
\begin{align}
    A \cdot T_u &= \frac{h}{e_u} \\
    e_u &=  m_u \frac{l_u^2}{T_u^2} \\
    T_u &= \sqrt{ \frac{ m_u l_u^2}{e_u} }\\
    h &= A e_u T_u \\
    &= A e_u \sqrt{ \frac{ m_u l_u^2}{e_u} }\\
    &= A  \sqrt{ e_u m_u l_u^2 } \\
    A &= \frac{h}{\sqrt{ e_u m_u l_u^2 }}
\end{align}
so we can clearly only select either the mass or the length to fix in a system of units which is self-consistent. 

## Angular Frequency Implication?
According to Bob Field's book "The Spectra and Dynamics of Diatomic Molecules: Revised and Enlarged Edition", he defines $\omega_e$ as:
\begin{align}
    \omega_e = \hbar \frac{1}{hc} \sqrt{\frac{k}{\mu}}
\end{align}

In [1]:
import math

import numpy as np
from scipy.special import gamma, genlaguerre

import scipy.integrate
import scipy.misc

import sympy.mpmath
import sympy.functions.combinatorial.factorials as fact
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.io import output_notebook, show
output_notebook()

/Users/joseph/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/joseph/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


Loading BokehJS ...

In [2]:
#PROGRAM CONSTANTS
STARTING_NUM_X_POINTS = 75
STARTING_NUM_STDEV_FOR_INTEGRATING = 5.0
DX_FACTOR_FOR_CONVERGENCE_CHECK = .8
DEFAULT_ZERO_TOLERANCE = 1.0E-5

ROOM_TEMPERATURE_KELVIN = 298.0

#UNIT CONSTANTS
kg_per_amu = 1.660468E-27
h_joules_seconds = 6.62607E-34 #Joule*seconds
hc_joules_centimeters = 1.9864458E-23
m_per_angstrom = 1.0E-10
k_B_wavenumbers_per_kelvin = .695035

In [3]:
#TEST DICTIONARIES
TEST1_SYSTEM_DICTIONARY = {"reduced_mass" : 1.0,
                      "alpha" : 1.0,
                      "center" : 0.0,
                      "D" : 2.0}
TEST2_SYSTEM_DICTIONARY = {"reduced_mass" : 1.0,
                      "alpha" : 1.5,
                      "center" : 0.5,
                      "D" : 1.0}

TEST_UNIVERSE_DICTIONARY = {"hbar" : 1.0 / (2.0 * np.pi),
                               "ZERO_TOLERANCE" : DEFAULT_ZERO_TOLERANCE,
                            "k_B" : 1.0}


In [4]:
#NITROGEN
Nitrogen_energy_scale_wavenumbers = 2358.57
Nitrogen_mass_scale_amu = 7.001537

Nitrogen_center_offset_angstroms = 1.097685
Nitrogen_length_scale_angstroms = 1.2203 - 1.097685

Nitrogen_energy_scale_joule = Nitrogen_energy_scale_wavenumbers * hc_joules_centimeters
Nitrogen_mass_scale_kg = Nitrogen_mass_scale_amu * kg_per_amu
Nitrogen_length_scale_m = Nitrogen_length_scale_angstroms * m_per_angstrom

h_nitrogen_units = h_joules_seconds / np.sqrt(Nitrogen_energy_scale_joule * Nitrogen_mass_scale_kg * Nitrogen_length_scale_m**2)
Nitrogen_time_scale_seconds = np.sqrt(Nitrogen_mass_scale_kg * Nitrogen_length_scale_m**2 / Nitrogen_energy_scale_joule)

k_B_Nitrogen_energy_per_K = k_B_wavenumbers_per_kelvin / Nitrogen_energy_scale_wavenumbers

Nitrogen_universe_dictionary = {"hbar" : h_nitrogen_units / (2.0 * np.pi),
                               "ZERO_TOLERANCE" : 1.0E-5,
                                "k_B" : k_B_Nitrogen_energy_per_K}
Nitrogen_Chi_1_Sigma_g_Plus = {"omega_e" : 2358.57 / Nitrogen_energy_scale_wavenumbers,
                               "chi_e_omega_e" : 14.324 / Nitrogen_energy_scale_wavenumbers,
                               "reduced_mass" : 7.001537 / Nitrogen_mass_scale_amu, 
                               "center" : (1.097685 - Nitrogen_center_offset_angstroms) / Nitrogen_length_scale_angstroms}
Nitrogen_a_1_Pi_g = {"omega_e" : 1694.208 / Nitrogen_energy_scale_wavenumbers,
                               "chi_e_omega_e" : 13.949 / Nitrogen_energy_scale_wavenumbers,
                               "reduced_mass" : 7.001537 / Nitrogen_mass_scale_amu, 
                               "center" : (1.2203 - Nitrogen_center_offset_angstroms) / Nitrogen_length_scale_angstroms}
print(h_nitrogen_units * Nitrogen_energy_scale_joule * Nitrogen_time_scale_seconds, h_joules_seconds)
assert(np.abs(h_nitrogen_units * Nitrogen_energy_scale_joule * Nitrogen_time_scale_seconds -h_joules_seconds) < DEFAULT_ZERO_TOLERANCE)

(6.6260699999999994e-34, 6.62607e-34)


In [19]:
class UnboundStateIndexError(Exception):
    def __init__(self):
        pass
class Morse(object):
    def __init__(self, system_dictionary , universe_dictionary ):
        
        #define the Universe
        self.hbar = universe_dictionary["hbar"]
        self.ZERO_TOLERANCE = universe_dictionary["ZERO_TOLERANCE"]
        self.k_B = universe_dictionary["k_B"]
        
        #define the system
        #terminology taken from Matsumoto and Iwamoto, 1993
        self.mu = system_dictionary["reduced_mass"]
        self.center = system_dictionary["center"]
        self.r = self.center
        
        if "alpha" in system_dictionary:
            self.alpha = system_dictionary["alpha"]
            self.D = system_dictionary["D"]

            self.omega_e = 2.0 * self.alpha * np.sqrt(self.D / (2.0 * self.mu))
            self.chi_e_omega_e = self.alpha**2 * self.hbar / (2.0 * self.mu)
        else:
            self.omega_e = system_dictionary["omega_e"]
            self.chi_e_omega_e = system_dictionary["chi_e_omega_e"]
            
            self.alpha = np.sqrt(2.0 * self.mu * self.chi_e_omega_e / self.hbar)
            self.D = 2.0 * self.mu *(self.omega_e / (2.0 * self.alpha))**2
            
        
        self.a = np.sqrt(2.0 * self.mu * self.D) / (self.alpha * self.hbar)
        self.maximum_index = int(np.floor(self.a - .5))
        
        #Harmonic Oscillator Approximation:
        k = self.potential_energy_gradientSquared(self.r)
        self.omega_HO = np.sqrt(k / self.mu)
        self.x0 = np.sqrt( self.hbar / (2.0 * self.omega_HO * self.mu))
        
        #determine the needed spatial parameters:
        self.index_to_xParams_dictionary = {}
        for energy_index in range(self.maximum_index + 1):
            #use the analytically calculated spread of the corresponding HO wavefunction to start guessing the needed spatial parameters
            HO_spatial_spread = self.x0 * np.sqrt(2 * energy_index + 1)
            
            x_min = self.r - STARTING_NUM_STDEV_FOR_INTEGRATING * HO_spatial_spread
            while np.abs(self.energy_eigenfunction_amplitude(energy_index, x_min)) > self.ZERO_TOLERANCE:
                x_min += -HO_spatial_spread
            x_max = self.r + STARTING_NUM_STDEV_FOR_INTEGRATING * HO_spatial_spread           
            while np.abs(self.energy_eigenfunction_amplitude(energy_index, x_max)) > self.ZERO_TOLERANCE:
                x_max += HO_spatial_spread
            
            keep_integrating = True
            
            number_x_points = STARTING_NUM_X_POINTS
            while keep_integrating:
                x_vals = np.linspace(x_min, x_max, number_x_points)
                psi_vals = self.energy_eigenfunction_amplitude(energy_index, x_vals)
                integral = scipy.integrate.simps(np.conj(psi_vals) * psi_vals, x = x_vals)
                if np.abs(integral - 1.0) < self.ZERO_TOLERANCE:
                    keep_integrating = False
                else:
                    number_x_points = number_x_points + 10
            dx = x_vals[1] - x_vals[0]
            self.index_to_xParams_dictionary[energy_index] = (x_min, x_max, number_x_points, dx)
            
            
        
    
    #POTENTIAL ENERGY STUFF:
    def potential_energy(self, x):
        return -2 * self.D * np.exp(- self.alpha * (x - self.r)) + self.D * np.exp(-2.0 * self.alpha * (x - self.r))
    
    def potential_energy_gradient(self, x):
        return 2.0 * self.alpha * self.D *(np.exp(- self.alpha * (x - self.r)) - np.exp(-2.0 * self.alpha * (x - self.r)))
        
    def potential_energy_gradientSquared(self, x):
        return 2.0 * self.alpha**2 * self.D *(-np.exp(- self.alpha * (x - self.r))  + 2.0 * np.exp(-2.0 * self.alpha * (x - self.r)))
    
    #ENERGY EIGENFUNCTION STUFF:
    def energy_eigenvalue(self, index):
        return -self.D + self.hbar * ( self.omega_e *(index + .5) - self.chi_e_omega_e *(index + .5)**2 )
    
    def energy_eigenfunction_amplitude(self, n, x):
        if n > self.maximum_index:
            raise UnboundStateIndexError()
        b_n = self.a - .5 - n
        N_n = np.sqrt(2.0 * self.alpha * b_n * scipy.misc.factorial(n) / gamma(2 * b_n + n + 1))
        z = 2.0 * self.a * np.exp(-self.alpha *(x - self.r))
        
        z_poly = np.power(z, b_n)
        z_exp = np.exp(-.5 * z)
        lag_part = genlaguerre(n, 2 * b_n)(z)
        
        return N_n * z_poly * z_exp * lag_part
    
    def x_values_for_energy_eigenfunction(self, n):
        x_min, x_max, num_x_points, dx = self.index_to_xParams_dictionary[n]
        return np.linspace(x_min, x_max, num_x_points)
    
    #THERMAL FUNCTIONS
    def partition_function_bound_state(self, T_Kelvin):
        energies = []
        for energy_index in range(self.maximum_index + 1):
            energies.append(self.energy_eigenvalue(energy_index))
        energies = np.array(energies)
        boltzman_factors = np.exp(-energies / (self.k_B * T_Kelvin))
        return np.sum(boltzman_factors)
    
    def thermal_weight_state_i(self, i, T_Kelvin):
        e_i = self.energy_eigenvalue(i)
        boltzmann_factor = np.exp(-e_i/(self.k_B * T_Kelvin))
        return boltzmann_factor / self.partition_function_bound_state(T_Kelvin)
    
class OffsetMorse(object):
    
    def __init__(self, ground_morse, excited_morse, universe_dictionary ):
        #define the Universe
        self.hbar = universe_dictionary["hbar"]
        self.ZERO_TOLERANCE = universe_dictionary["ZERO_TOLERANCE"]
        
        #assign variables
        self.ground_morse = ground_morse
        self.excited_morse = excited_morse
        
        self.franck_condon_factors = np.zeros((self.ground_morse.maximum_index + 1, self.excited_morse.maximum_index + 1))
        for ground_index in range(self.ground_morse.maximum_index + 1):
            #get the ground state's needed x parameters
            ground_xMin, ground_xMax, ground_numPoints, ground_dx = self.ground_morse.index_to_xParams_dictionary[ground_index]
            
            for excited_index in range(self.excited_morse.maximum_index + 1 ):
                #get the excited state's needed x parameters
                excited_xMin, excited_xMax, excited_numPoints, excited_dx = self.excited_morse.index_to_xParams_dictionary[excited_index]
                
                #make best guess for needed x values for integrating the overlap
                x_min = min([ground_xMin, excited_xMin])
                x_max = max([excited_xMax, ground_xMax])
                dx_needed = min([ground_dx, excited_dx]) 
                
                new_dx = 1.0 / (1.0/ground_dx + 1.0/excited_dx)
                
                #integrate once
                x_vals = np.arange(x_min, x_max, new_dx)
                g_func_vals = self.ground_morse.energy_eigenfunction_amplitude(ground_index, x_vals)
                e_func_vals = self.excited_morse.energy_eigenfunction_amplitude(excited_index, x_vals)
                amp_to_integrate = e_func_vals * np.conj(g_func_vals)
                old_integral = scipy.integrate.simps(amp_to_integrate, x= x_vals)
                
                if np.isnan(old_integral):
                    for index, data in enumerate(amp_to_integrate):
                        if np.isnan(data):
                            amp_to_integrate[index] = 0.0
                    old_integral = scipy.integrate.simps(amp_to_integrate, x= x_vals)
                    nan_issue_encountered = True
                else:
                    nan_issue_encountered = False
                    
                
                #check to make sure integral is converged
                keep_integrating = True
                number_failures = 0
                max_failures = 5
                calculated_integrals = [old_integral]
                while keep_integrating:
                    dx_needed = dx_needed * DX_FACTOR_FOR_CONVERGENCE_CHECK
                    
                    x_vals = np.arange(x_min, x_max, dx_needed)
                    
                    g_func_vals = self.ground_morse.energy_eigenfunction_amplitude(ground_index, x_vals)
                    e_func_vals = self.excited_morse.energy_eigenfunction_amplitude(excited_index, x_vals)
                    
                    amp_to_integrate = e_func_vals * np.conj(g_func_vals)
                    new_integral = scipy.integrate.simps(amp_to_integrate, x = x_vals)
                     
                    
                    if np.isnan(new_integral):
                        for index, data in enumerate(amp_to_integrate):
                            if np.isnan(data):
                                amp_to_integrate[index] = 0.0
                        new_integral = scipy.integrate.simps(amp_to_integrate, x= x_vals)
                    
                    err = np.abs((new_integral - old_integral) / new_integral) 
                    
                    calculated_integrals.append(new_integral) 
                    
                    if  err < self.ZERO_TOLERANCE:
                        keep_integrating = False
                    else:
                        number_failures += 1
                        old_integral = new_integral
                    if number_failures > max_failures:
                        #It may just be that we're at zero and that's why we're unconverged
                        if new_integral / np.max(self.franck_condon_factors) < self.ZERO_TOLERANCE:
                            keep_integrating = False
                        else:
                            print(ground_index, excited_index)
                            print( amp_to_integrate)
                            plt.plot(calculated_integrals)
                            plt.figure()
                            plt.plot(amp_to_integrate)
                            raise Exception()
                
                self.franck_condon_factors[ground_index, excited_index] = new_integral
    def stick_absorption_spectrum(self, starting_ground_index):
        relevant_FCFs = self.franck_condon_factors[starting_ground_index,:]
        frequency_values = []
        ground_energy = self.ground_morse.energy_eigenvalue(starting_ground_index)
        for excited_index in range(self.excited_morse.maximum_index + 1):
            energy_gap = self.excited_morse.energy_eigenvalue(excited_index) - ground_energy
            frequency_values.append(energy_gap / self.hbar)
        
        return frequency_values, np.abs(relevant_FCFs)**2
    
    def total_absorption_spectrum(self, T_Kelvin):
        all_frequencies = []
        all_peaks = []
        for i in range(ground.maximum_index + 1):
            w, I = Nitrogen_offsetMorse.stick_absorption_spectrum(i)
            thermal_weight = ground.thermal_weight_state_i(i, ROOM_TEMPERATURE_KELVIN)
            spectrum = thermal_weight * I
            all_frequencies.append(w)
            all_peaks.append(spectrum)
        all_frequencies = np.concatenate(all_frequencies)
        all_peaks = np.concatenate(all_peaks)
        return all_frequencies, all_peaks

        

In [20]:
ground = Morse(system_dictionary=Nitrogen_Chi_1_Sigma_g_Plus, universe_dictionary = Nitrogen_universe_dictionary)
excited = Morse(system_dictionary=Nitrogen_a_1_Pi_g, universe_dictionary = Nitrogen_universe_dictionary)

In [21]:
Nitrogen_offsetMorse = OffsetMorse(ground_morse = ground, excited_morse = excited, universe_dictionary = Nitrogen_universe_dictionary)


In [ ]:
plt.figure()
# for ground_i in range(ground.maximum_index):
for ground_i in [0]:
    x_vals = ground.x_values_for_energy_eigenfunction(ground_i)
    f_vals = ground.energy_eigenfunction_amplitude(ground_i, x_vals)
    plt.plot(x_vals, np.real(f_vals))
x_vals_g = x_vals
# plt.xlim(0, .0002)
# plt.figure()
for excited_i in [33]:
    x_vals = excited.x_values_for_energy_eigenfunction(excited_i)
    f_vals = excited.energy_eigenfunction_amplitude(excited_i, x_vals)
    plt.plot(x_vals, np.real(f_vals))
# plt.xlim(0, .0002)